In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
from wdtools import *

In [19]:
# this can take about 9 minutes to read from 2009 to 2023
start = time.time()
all_taxlot = combine_taxlot(yearstart=2009, yearend=2017)
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

2009
2011
2014
2015
2016
it took 11.0 minutes to complete


In [4]:
all_taxlot.shape

(8378764, 3)

In [20]:
n=7

In [6]:
wd_df = get_all_wd(num=n-1)

In [8]:
wd_df.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot', 'SetID'],
      dtype='object')

In [9]:
wd_df.shape

(4972, 26)

In [7]:
setID = 'Set00'+str(n); nm_to_add = wd_df.shape[0]

In [8]:
wd = combine_wd_tables(setID=setID, nm_to_add=nm_to_add)

In [27]:
wd[wd.wetdet_delin_number == 'WD2012-0170']

,wetdet_delin_number,trsqq,parcel_id,address_location_desc,city,county,site_name,site_desc,latitude,longitude,...,Coord-Source,response_date,reissuance_response_date,project_id,site_id,record_ID,notes,recyear,IDyear,missinglot
344,WD2012-0170,15S10E00,1003,"West McKinney Butte Rd, Sisters",Sisters,Deschutes,Sisters Skatepark Proj 15S10E00,WD2012-0170 Sisters Skatepark Project,44.2958,-121.5796,...,DSL Verified,2012-05-10,NaT,NaN,85306,5317,,2012,2012,N


In [12]:
wd.county.unique()

array(['Clatsop', 'CLATSOP', 'Deschutes', 'DESCHUTES'], dtype=object)

In [13]:
wd.loc[:, 'county'] = wd.county.apply(lambda x: x.capitalize())

In [14]:
wd.groupby(['county'])['wetdet_delin_number'].count()

county
Clatsop      296
Deschutes    134
Name: wetdet_delin_number, dtype: int64

In [15]:
wd.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'DocumentName', 'DecisionLink', 'is_batch_file', 'status_name',
       'received_date', 'Coord-Source', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot'],
      dtype='object')

In [16]:
wd.trsqq.apply(lambda x: scan_trsqq(x))

0      0
1      0
2      0
3      0
4      0
      ..
425    0
426    0
427    0
428    0
429    0
Name: trsqq, Length: 430, dtype: int64

wd[wd.trsqq=='07N0908C'] # manually corrected it in the same table

url = wd.loc[wd.wetdet_delin_number == 'WD2011-0047', 'DecisionLink'].values[0]
if str(url) == 'nan':
    print('Decision link is not available')
else:
    webbrowser.open(url)

In [21]:
wdf, setgdf, unmatched_df = run_Tier1(setID=setID, nm_to_add=nm_to_add, all_taxlot=all_taxlot)

it took 0.0 minutes to complete
it is about 14.88% of data in the original 430 records unmatched
there are 0 records (0.0% of the original records) without parcel id


In [10]:
wdf.shape

(430, 25)

In [19]:
unmatched_df.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'DocumentName', 'DecisionLink', 'is_batch_file', 'status_name',
       'received_date', 'Coord-Source', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot'],
      dtype='object')

In [11]:
unmatched_df.shape

(64, 25)

In [12]:
len(unmatched_df.wetdet_delin_number.unique())

33

In [22]:
setgdf.columns

Index(['wdID', 'trsqq', 'parcel_id', 'loc_desc', 'city', 'county', 'site_name',
       'site_desc', 'latitude', 'longitude', 'doc_name', 'doc_link',
       'isbatfile', 'status_nm', 'receiveddt', 'coord_src', 'responsedt',
       'reissuance', 'project_id', 'site_id', 'record_ID', 'notes', 'recyear',
       'IDyear', 'missinglot', 'lots', 'lot', 'cnt_code', 'ORTaxlot',
       'geometry'],
      dtype='object')

In [13]:
setgdf.shape

(686, 30)

In [14]:
len(setgdf.wdID.unique())

279

In [22]:
r1_df, r2_df = run_Tier2_step1(setID=setID, unmatched_df=unmatched_df, all_taxlot=all_taxlot)

reviewing WD2015-0449
all lots are matched...
corrected trsqq...
reviewing WD2015-0283
all lots are matched...
corrected trsqq...
reviewing WD2008-0294
trsqq matched, checking county code...
county code is corrected, need to check lot numbers...
lot numbers might be wrong...
reviewing WD2008-0343
there is not any matched lot, need to review trsqq, the close-match is 08N10W27DD
lots to check: ['202'], and lots to compare: ['100', '102', '200', '300', '400', '500', '600', '700', '800', '900', '1000', '1100', '1200', '1300', '1400', '1500', '1700', '1800', '1900', '2000', '2100', '2200', '2300', '2400', '2500', '2600', '2700', '2800', '2900', '3000', '3100', '3200', '3203', '3300', '3301', '3400', '3401', '3500', '3600', '3700', '3800', '3900', '4000', '4100', '4200', '4300', '4400', '4500', '4600', '5000', '5400', '5401', '5800', '5900', '6000', '6100', '6200', '6300', '6400', '6500', '6600', '6700', '6800', '6900', '7000', '7100', '7200', '7300', '7400', '7500', '7600', '7700', 'ROADS']

Manual run notes_review_set7 to get correction notes

In [23]:
matched, unmatched_df2 = run_Tier2_step3(r1_df=r1_df, r2_df=r2_df, setID=setID, nm_to_add=nm_to_add, 
                                        wd=wdf, all_taxlot=all_taxlot)

it is about 12.09% of data in the original 430 records unmatched
there are 0 records (0.0% of the original records) without parcel id
